<a href="https://colab.research.google.com/github/tonyyoon123/Boolean-Model_Final/blob/master/Final_project(NLP_datacleaning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import csv
import pandas as pd
import pickle
nltk.download('punkt')

**Import Text Files and save it to documents.**

In [ ]:
D = {}
for x  in range(11):
    x=x+1
    file=str(x)+".txt"
    D[x]=open(file).read()

In [ ]:
scripts=""
for x in range(11):
    x=x+1
    scripts=scripts+" \n"+D[x]


In [ ]:
D.keys()

**Tokenize the Texts**

In [ ]:
next(iter(D.keys()))

In [ ]:
next(iter(D.values()))

In [ ]:
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [ ]:
D_combined = {key: [combine_text(value)] for (key, value) in D.items()}

In [ ]:
pd.set_option('max_colwidth',150)
D_df = pd.DataFrame.from_dict(D_combined).transpose()
D_df.columns = ['script']
D_df = D_df.sort_index()
D_df

In [ ]:
D_df.script.loc[1]

In [ ]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
D_clean = pd.DataFrame(D_df.script.apply(round1))
D_clean

In [ ]:
D_clean.script.loc[1]

In [ ]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
D_clean = pd.DataFrame(D_clean.script.apply(round2))
D_clean

In [ ]:
D_clean.script.loc[1]

# **Organize the data**

In [ ]:
D_df

In [ ]:
D_clean.to_pickle("corpus.pkl")

In [ ]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [ ]:
cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words='english', ngram_range=(2,2), analyzer='word')
D_cv = cv.fit_transform(D_clean.script)
D_dtm  = pd.DataFrame(D_cv.toarray(), columns=cv.get_feature_names())
D_dtm.index = D_clean.index
D_dtm

In [ ]:
D_dtm.to_pickle("dtm.pkl")

In [ ]:
D_clean.to_pickle('D_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))